In [1]:
import boto3,os
import sagemaker as sage
from sagemaker import get_execution_role
from time import gmtime,strftime

In [2]:
region = boto3.session.Session().region_name
account_id = boto3.client('sts').get_caller_identity().get('Account')
ecr_repository = 'sagemaker-nmf-container'
tag = ':nmflatest'
training_repository_uri = f'{account_id}.dkr.ecr.{region}.amazonaws.com/{ecr_repository}'

role = get_execution_role()
sess = sage.Session()

In [3]:
!docker build -t $ecr_repository .

Sending build context to Docker daemon  65.54kB
Step 1/10 : FROM python:3.7-slim-buster
3.7-slim-buster: Pulling from library/python

5a41d630: Pulling fs layer 
8b7b4c32: Pulling fs layer 
a3eb7bae: Pulling fs layer 
e182f49d: Pulling fs layer 
7d8ed69c: Pull complete 2.6MB/2.6MBMBBDigest: sha256:5375725c3c0a0215279c1c5ddb33f91d31f0eb37010140397e5c7e5530073d2c
Status: Downloaded newer image for python:3.7-slim-buster
 ---> e7d86653f62f
Step 2/10 : MAINTAINER ASC "mbod@asc.upenn.edu jmparelman@gmail.com"
 ---> Running in 73618c967580
Removing intermediate container 73618c967580
 ---> d66f1655228d
Step 3/10 : RUN mkdir -p /data/input /data/output
 ---> Running in a7f62428e6eb
Removing intermediate container a7f62428e6eb
 ---> 82d126aaf150
Step 4/10 : RUN apt-get -y update && apt-get install -y --no-install-recommends         build-essential         python-dev         python3-pip     && pip3 install -U pip
 ---> Running in 631e357be8e5
Get:1 http://deb.debian.org/debian buster InRelease 

In [4]:
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 064258348567.dkr.ecr.us-east-1.amazonaws.com

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [5]:
!aws ecr create-repository --repository-name $ecr_repository


An error occurred (RepositoryAlreadyExistsException) when calling the CreateRepository operation: The repository with name 'sagemaker-nmf-container' already exists in the registry with id '064258348567'


In [6]:
!docker tag {ecr_repository} "064258348567.dkr.ecr.us-east-1.amazonaws.com/sagemaker-nmf-container:latest"
!docker push "064258348567.dkr.ecr.us-east-1.amazonaws.com/sagemaker-nmf-container:latest"

The push refers to repository [064258348567.dkr.ecr.us-east-1.amazonaws.com/sagemaker-nmf-container]

de7c8065: Preparing 
aa32d33f: Preparing 
6f6f24c2: Preparing 
d72f0b50: Preparing 
3c6439bc: Preparing 
6bb9ffea: Preparing 
663e622b: Preparing 
845af46d: Preparing 
aa32d33f: Pushed   368.8MB/362.7MBAlatest: digest: sha256:35834a517f2861632c4442889128eebe70dc3383cdec366b658fe368287eacab size: 2211


In [7]:
nmf = sage.estimator.Estimator( "064258348567.dkr.ecr.us-east-1.amazonaws.com/sagemaker-nmf-container:latest",
                              role,1, 'ml.m5.4xlarge',
                              output_path = 's3://ascsagemaker/JMP_congressional_nmf/unigram_models',
                              sagemaker_session=sess,
                              max_run=9000)

# START JOBS

In [8]:
congress_list = range(97,115)
for chamber in congress_list:
    cstr = '{:0>3}'.format(chamber)
    print(cstr)

097
098
099
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114


In [9]:
for chamber in congress_list:
    cstr = '{:0>3}'.format(chamber)
    print('Scheduling', chamber)
    nmf.fit(f's3://ascsagemaker/JMP_congressional_nmf/unigram_dtms/{cstr}.pkl',wait=False)

Scheduling 97
Scheduling 98
Scheduling 99
Scheduling 100
Scheduling 101
Scheduling 102
Scheduling 103
Scheduling 104
Scheduling 105
Scheduling 106
Scheduling 107
Scheduling 108
Scheduling 109
Scheduling 110
Scheduling 111
Scheduling 112
Scheduling 113
Scheduling 114
